In [3]:
# Imports
import os
import sys
import glob
import torch
import torchvision

import numpy as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets, models
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser

%matplotlib inline


In [4]:
data_path = "../input/ammi-2021-convnets/"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")

In [5]:
import os
os.environ['KAGGLE_USERNAME'] = 'muhirwasalomon'
os.environ['KAGGLE_KEY'] = 'f2e155be15e51da2a38317f327c0669b'
!kaggle competitions download -c ammi-2023-convnets --force
!unzip -q ammi-2023-convnets.zip -d ammi-2023-convnets


100%|██████████████████████████████████████▊| 2.29G/2.30G [00:10<00:00, 226MB/s]
100%|███████████████████████████████████████| 2.30G/2.30G [00:10<00:00, 232MB/s]


In [6]:
train_path = 'ammi-2023-convnets/train/train'
test_path = 'ammi-2023-convnets/test/test'


In [74]:
import torch
import torchvision.transforms as transforms
from sklearn.utils import class_weight
from collections import Counter
torch.random.seed()
# Define the transformations
train_transform = transforms.Compose([
    transforms.CenterCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Compute class weights
def compute_class_weights(labels):
    label_counts = Counter(labels)
    class_weights = [1.0 / label_counts[label] for label in labels]
    return torch.tensor(class_weights, dtype=torch.float)

class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, fileName])
        self.file_list = files

        # Compute class weights
        labels = [item[0] for item in self.file_list]
        self.class_weights = compute_class_weights(labels)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][1]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)

        if self.transform:
            im = self.transform(im)

        return im, classCategory


In [9]:
train_data = CassavaDataset(train_path, transform=train_transform)
test_data = CassavaDataset(test_path, transform=test_transform)

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

validation_split = 0.2
random_seed = 42
shuffle_dataset = True  # Define shuffle_dataset variable here

# Creating data indices for training and validation splits
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]


In [63]:
batch_size=4

In [64]:
from torch.utils.data import SubsetRandomSampler, DataLoader

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(train_data, batch_size=batch_size, sampler=val_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [65]:

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [66]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision.models import resnet50

# # Define the ResNet-based model
# class ResNetClassifier(nn.Module):
#     def __init__(self, num_classes, weight_decay=0.000001):
#         super(ResNetClassifier, self).__init__()
#         # Load the pre-trained ResNet50 model
#         self.resnet = resnet18(pretrained=True)
        
#         # Modify the last fully-connected layer for the desired number of classes
#         num_features = self.resnet.fc.in_features
#         self.resnet.fc = nn.Linear(num_features, num_classes)
        
#         # Define the weight decay (L2 regularization) for the parameters
#         self.weight_decay = weight_decay

#     def forward(self, input):
#         x = self.resnet(input)
#         return x

# model = ResNetClassifier(num_classes=5)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.RMSprop(model.parameters(), lr=0.0002, weight_decay=model.weight_decay)


In [67]:
!pip install efficientnet_pytorch


In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import densenet

# Define the DenseNet-based model
class DenseNetClassifier(nn.Module):
    def __init__(self, num_classes, weight_decay=0.000001):
        super(DenseNetClassifier, self).__init__()
        # Load the pre-trained DenseNet model
        self.densenet = densenet.densenet121(pretrained=True)
        
        # Modify the last fully-connected layer for the desired number of classes
        num_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_features, num_classes)
        
        # Define the weight decay (L2 regularization) for the parameters
        self.weight_decay = weight_decay

    def forward(self, input):
        x = self.densenet(input)
        return x

model = DenseNetClassifier(num_classes=5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0002, weight_decay=model.weight_decay)


Loaded pretrained weights for efficientnet-b0


In [70]:
def train(model, criterion, train_data, optimizer, num_epochs):
    """Train a model without cross-validation."""

    ema_loss = None

    print('----- Training Loop -----')

    # Create data loader for training data.
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Loop over epochs.
    for epoch in range(num_epochs):
        # Training phase
        model.train()

        # Loop over training data.
        for batch_idx, (features, target) in enumerate(train_loader):
            # Move data to the device.
            features = features.to(device)
            target = target.to(device)

            # Forward pass.
            output = model(features)
            loss = criterion(output, target)

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update exponential moving average of the loss.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01

        # Print training progress at the end of the epoch.
        print('Epoch: {} \tTraining Loss: {:.3f}'.format(epoch, ema_loss))


In [71]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0
    print('----- Model Evaluation -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():

        # Loop over test data.
        for features, target in data_loader:

            # Move data to the device.
            features = features.to(device)
            target = target.to(device)

            # Forward pass.
            output = model(features)

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            # Move tensors to the same device as the model.
            pred = pred.to(device)
            target = target.to(device)

            # Count number of correct predictions.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100. * correct / len(val_sampler)
    print(f'Test accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)')
    torch.save(model.state_dict(), 'model.ckpt')
    return percent


In [72]:
# def test(model, data_loader):
#     """Measures the accuracy of a model on a data set."""
#     # Make sure the model is in evaluation mode.
#     model.eval()
#     correct = 0
#     total = 0

#     print('----- Model Evaluation -----')
#     # We do not need to maintain intermediate activations while testing.
#     with torch.no_grad():
#         # Loop over test data.
#         for features, target in data_loader:
#             # Move data to the device.
#             features = features.to(device)
#             target = target.to(device)

#             # Forward pass.
#             output = model(features)

#             # Get the label corresponding to the highest predicted probability.
#             _, predicted = torch.max(output.data, 1)

#             # Count number of correct predictions.
#             total += target.size(0)
#             correct += (predicted == target).sum().item()

#     # Calculate test accuracy.
#     accuracy = 100 * correct / total
#     print(f'Test accuracy: {correct} / {total} ({accuracy:.2f}%)')

#     return accuracy


In [73]:
# Define the number of epochs and patience
num_epochs = 10
patience = 3

# Train the model 
train(model, criterion, train_data, optimizer, num_epochs)

# Evaluate the model on the validation set
val_accuracy = test(model, val_loader)


----- Training Loop -----
Epoch: 0 	Training Loss: 0.931
Epoch: 1 	Training Loss: 0.823
Epoch: 2 	Training Loss: 0.725
Epoch: 3 	Training Loss: 0.708
Epoch: 4 	Training Loss: 0.637
Epoch: 5 	Training Loss: 0.629
Epoch: 6 	Training Loss: 0.671
Epoch: 7 	Training Loss: 0.613
Epoch: 8 	Training Loss: 0.535
Epoch: 9 	Training Loss: 0.569
----- Model Evaluation -----
Test accuracy: 957 / 5656 (85%)


In [ ]:

# test(model, test_loader)

In [61]:
import os
from os import listdir
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms

# Step 1
test_directory = "ammi-2023-convnets/test/test/0"
predictions, test_image_fileName = [], []

# Step 2
# Process image
def preprocess_image(image):
    image = image.resize((224, 224))
    image = transforms.ToTensor()(image)
    image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
    return image

# Step 3
def predict(image, model):
    # Pass the image through the model
    output = model(image.to(device))
    # Reverse the log function of the output
    output = torch.exp(output)
    # Get the predicted class
    _, predicted_class = output.max(1)
    return predicted_class.item()

# Step 4
# Get the list of class names
class_names = train_loader.dataset.classes

# Step 5
try:
    # Iterate over the test images
    test_images = listdir(test_directory)
    for image_name in test_images:
        # Check if the item is not a directory
        if not os.path.isdir(os.path.join(test_directory, image_name)):
            # Get the image path
            image_path = os.path.join(test_directory, image_name)
            print(f"Preprocess image: {image_path}")
            
            # Open and preprocess the image
            image = Image.open(image_path)
            image = preprocess_image(image)
            image = image.unsqueeze(0).to(device)
            
            # Perform the prediction on the image
            with torch.no_grad():
                top_class = predict(image, model)
                # Add the predicted class and image name to the list
                predictions.append(class_names[top_class])
                test_image_fileName.append(image_name)
                # Print the prediction result for the image
                print(f"Prediction for image {image_path}: {class_names[top_class]}")

    # Sort predictions and image filenames based on predictions
    sorted_predictions, sorted_filenames = zip(*sorted(zip(predictions, test_image_fileName)))
    predictions = list(sorted_predictions)
    test_image_fileName = list(sorted_filenames)

except Exception as e:
    # Handle any exceptions that occur during the process
    print(e)


Preprocess image: ammi-2023-convnets/test/test/0/test-img-37.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-37.jpg: cgm
Preprocess image: ammi-2023-convnets/test/test/0/test-img-3527.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-3527.jpg: healthy
Preprocess image: ammi-2023-convnets/test/test/0/test-img-2206.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-2206.jpg: cmd
Preprocess image: ammi-2023-convnets/test/test/0/test-img-2515.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-2515.jpg: cmd
Preprocess image: ammi-2023-convnets/test/test/0/test-img-2197.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-2197.jpg: cmd
Preprocess image: ammi-2023-convnets/test/test/0/test-img-302.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-302.jpg: cgm
Preprocess image: ammi-2023-convnets/test/test/0/test-img-2380.jpg
Prediction for image ammi-2023-convnets/test/test/0/test-img-2380.jpg: cbsd
Preproc

In [ ]:
# Make submission here

In [62]:
import pandas as pd

submission_data = {"Category": predictions, "Id": test_image_fileName}
submission_data_frame = pd.DataFrame(submission_data)

# Save the DataFrame to CSV
submission_data_frame.to_csv('/kaggle/working/submission_file8.csv', index=False)


In [ ]:
import pandas as pd

submission_data = {"Category": predictions, "Id": test_image_fileName}
submission_data_frame = pd.DataFrame(submission_data)

# Save the DataFrame to CSV
submission_data_frame.to_csv('/kaggle/working/submission_file4.csv', index=False)
